In [ ]:
# setting up drive and importing libraries
import tensorflow
import cv2
import os
from google.colab import drive

In [ ]:
drive.mount("/content/drive")

In [ ]:
cd / content / drive / MyDrive / medical - images

In [ ]:
# data preprocessing
path = "/content/drive/MyDrive/medical-images/"
categories = os.listdir(path)
labels = [label for label in range(len(categories))]
label_names = dict(zip(categories, labels))
print(label_names)

In [ ]:
img_size = 256
data = []
label = []
for category in categories:
    folder_path = os.path.join(path, category)
    img_names = os.listdir(folder_path)

    for img_name in img_names:
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)
        try:
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            resized = cv2.resize(gray, (img_size, img_size))
            data.append(resized)
            label.append(label_names[category])
        except Exception as e:
            print("Exception:", e)

In [ ]:
import numpy as np

data = np.array(data) / 255.0
data = np.reshape(data, (data.shape[0], size, size, 1))
label = np.array(label)
from keras.utils import np_utils

new_label = np_utils.to_categorical(label)

In [ ]:
new_label.shape

In [ ]:
# CNN
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.callbacks import ModelCheckpoint

model = Sequential()

model.add(Conv2D(128, (3, 3), input_shape=data.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dropout(0.2))

model.add(Dense(128, activation="relu"))

model.add(Dropout(0.1))
model.add(Dense(64, activation="relu"))

model.add(Dense(5, activation="softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train-test split
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data, new_label, test_size=0.1)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for i in range(20):
    plt.subplot(5, 5, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(np.squeeze(x_test[i]))
    plt.xlabel(categories[np.argmax(y_test[i])])
plt.show()

In [ ]:
history = model.fit(x_train, y_train, epochs=11, validation_split=0.2)

In [ ]:
model.save("model.h5")

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
vaL_loss, val_accuracy = model.evaluate(x_test, y_test, verbose=0)
print("test loss:", vaL_loss, "%")
print("test accuracy:", val_accuracy, "%")

In [ ]:
# confusion matrix
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix

test_labels = np.argmax(y_test, axis=1)
predictions = model.predict(x_test)
predictions = np.argmax(predictions, axis=-1)
cm = confusion_matrix(test_labels, predictions)
plt.figure()
plot_confusion_matrix(cm, figsize=(12, 8), hide_ticks=True, cmap=plt.cm.Oranges)
plt.xticks(range(5), ["Normal", "Doubtful", "Mid", "Moderate", "Severe"])
plt.yticks(range(5), ["Normal", "Doubtful", "Mid", "Moderate", "Severe"])
plt.show()